In [31]:
#import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")

In [32]:
using JuMP, GLPK
using DataFrames
using CSV

In [33]:
#load cost data
con_generation = CSV.File("../data/conventional_generators.csv") |> DataFrame

#load wind_technicaldata
wind_generation = CSV.File("../data/wind_farms.csv") |> DataFrame

# load wind profile
wind_profile = CSV.File("../data/wind_powerprofile_200.csv") |> DataFrame


Row,Hour,W1,W2,W3,W4,W5,W6
,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,76.8921,101.54,92.8003,95.3709,96.002,70.9073
2,2,66.8277,90.9989,109.167,107.937,103.616,112.618
3,3,78.422,116.959,142.88,134.78,128.276,133.832
4,4,64.1437,134.249,159.57,133.664,142.969,154.233
5,5,102.22,145.507,160.944,165.303,144.03,161.49
6,6,134.039,131.112,157.023,161.828,144.636,155.222
7,7,146.517,153.67,141.566,159.927,152.86,153.604
8,8,143.176,163.392,155.294,170.805,158.643,164.307
9,9,163.297,144.48,172.638,185.318,150.901,166.558


In [34]:
#demand bids
name = "demand_hour_0.csv"
demand_bids = CSV.File("../data/demand_bids_hour/" * name) |> DataFrame
 
# number of convential generators
 G = size(con_generation, 1)

 # number of demand
 D = size(demand_bids, 1)

 # number of wind generators
 W = size(wind_generation, 1)

4

In [35]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[], objective_value = Float64[])

# For x variables, manually add each column. This is a one-time setup.
for i in 1:G
    result_df[!, Symbol("x_con$i")] = Float64[]
end

# For w variables, manually add each column. This is a one-time setup.
for i in 1:W
    result_df[!, Symbol("x_wind$i")] = Float64[]
end

# For y variables, manually add each column. This is a one-time setup.
for i in 1:D
    result_df[!, Symbol("y$i")] = Float64[]
end

In [36]:

for hour in 1:24
  # Create a new model with GLPK solver
  model = Model(GLPK.Optimizer)

  # Define the decision variables for every generator
  unregister(model, :x)  # Unregister the existing variable named "x" from the model
  
  #demand bids
  name = "demand_hour_" * string(hour-1) * ".csv"
  demand_bids = CSV.File("../data/demand_bids_hour/" * name) |> DataFrame

   # number of demand
   D = size(demand_bids, 1)

   #@variable(model, x_bin[1:G], Bin)  # binary variable for on/off status
   @variable(model, x_con[1:G])  # power output variable

   for g in 1:G
    set_lower_bound(x_con[g], 0)  # Set the lower bound
    set_upper_bound(x_con[g], con_generation[g,6])  # Set the upper bound
   end

   # Add constraints for each plant
  #for g in 1:G
  #  @constraint(model, x_con[g] >= capacity[g,4] * x_bin[g])  # If plant i is on, power must be at least 15
  #  @constraint(model, x_con[g] <= capacity[g,3] * x_bin[g])  # If plant i is on, power cannot exceed 20
  #end

  # wind decision variables
  @variable(model, x_wind[1:W])  # Create the new variable
  for g in 1:W
      set_lower_bound(x_wind[g], 0)  # Set the lower bound
      set_upper_bound(x_wind[g], wind_profile[hour,g+1])  # Set the upper bound
  end

  # decision variables for demand
  @variable(model, y[1:size(demand_bids, 1)])  # Create the new variable
  for bid in 1:D
      set_lower_bound(y[bid], 0)  # Set the lower bound
      set_upper_bound(y[bid], demand_bids[bid,2])  # Set the upper bound
  end


  # Define the objective function
  @objective(model, Max, sum(demand_bids[d,3] * y[d] for d in 1:D) - sum(con_generation[g,3] * x_con[g] for g in 1:G))

  # Add the constraint for the balance between supply and demand
  @constraint(model, BalanceConstraint,  sum(x_con[g] for g in 1:G) + sum(x_wind[w] for w in 1:W) - sum(y[d] for d in 1:D) == 0)

  # Print the model
  #print(model)

  # Solve the model
  optimize!(model)

  # Check the status of the solution
  status = termination_status(model)
  if status == MOI.OPTIMAL
      println("Hour ", hour, ": Optimal solution found")
      #println("Objective value: ", objective_value(model))
      #println("x values: ", [value(x[i]) for i in 1:G])
      #println("x_wind values: ", [value(x_wind[i]) for i in 1:W])
      #println("y values: ", [value(y[i]) for i in 1:D])
      println(dual(BalanceConstraint))

      #save results to dataframe
      push!(result_df, [hour, objective_value(model), [value(x_con[i]) for i in 1:G]..., [value(x_wind[i]) for i in 1:W]..., [value(y[i]) for i in 1:D]...])

  elseif status == MOI.INFEASIBLE
      println("Hour ", hour, ": No feasible solution found")
  else
      println("Hour ", hour, ": Solver status: ", status)
  end
end

Hour 1: Optimal solution found
10.52
Hour 2: Optimal solution found
10.26905187546268
Hour 3: Optimal solution found
10.52
Hour 4: Optimal solution found
7.0
Hour 5: Optimal solution found
7.0


Hour 6: Optimal solution found
8.527328469352174
Hour 7: Optimal solution found
10.512712364322477
Hour 8: Optimal solution found
9.008746858513351
Hour 9: Optimal solution found
7.730244978885052
Hour 10: Optimal solution found
10.52
Hour 11: Optimal solution found
10.52
Hour 12: Optimal solution found
10.52
Hour 13: Optimal solution found
10.89
Hour 14: Optimal solution found
10.89
Hour 15: Optimal solution found
11.25296892824927
Hour 16: Optimal solution found
10.52
Hour 17: Optimal solution found
10.52
Hour 18: Optimal solution found
13.32
Hour 19: Optimal solution found
10.52
Hour 20: Optimal solution found
11.813749279032312
Hour 21: Optimal solution found
7.0
Hour 22: Optimal solution found
10.52
Hour 23: Optimal solution found
10.52
Hour 24: Optimal solution found
7.0


In [37]:
result_df

Row,hour,objective_value,x_con1,x_con2,x_con3,x_con4,x_con5,x_con6,x_con7,x_con8,x_con9,x_con10,x_con11,x_con12,x_wind1,x_wind2,x_wind3,x_wind4,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,21854.3,0.0,0.0,0.0,0.0,0.0,108.5,108.5,280.0,280.0,210.0,40.3736,0.0,76.8921,101.54,92.8003,95.3709,67.8413,0.0,165.565,0.0,197.87,62.1878,208.37,113.876,80.7634,112.261,46.8428,44.4199,0.0,78.3405,106.608,109.031,0.0
2,2,19674.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,280.0,280.0,210.0,0.0,0.0,66.8277,90.9989,109.167,107.937,63.7911,0.0,78.7092,0.0,186.057,58.4751,195.93,0.0,0.0,105.559,44.0462,41.768,80.4982,73.6635,0.0,102.521,113.913
3,3,29052.6,0.0,0.0,0.0,0.0,0.0,108.5,102.2,280.0,280.0,210.0,0.0,0.0,78.422,116.959,142.88,134.78,60.7534,54.2441,148.267,108.488,177.197,55.6906,186.6,101.979,0.0,100.532,41.9488,39.779,76.665,0.0,95.4696,97.6394,108.488
4,4,16618.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,280.0,280.0,163.815,0.0,0.0,64.1437,134.249,159.57,133.664,59.7408,0.0,145.796,106.68,174.244,0.0,183.49,0.0,71.12,98.8569,41.2496,0.0,75.3872,68.9864,93.8785,96.0121,0.0
5,5,19401.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,280.0,280.0,186.014,0.0,0.0,102.22,145.507,160.944,165.303,59.7408,53.34,145.796,106.68,0.0,54.7624,183.49,100.279,71.12,98.8569,41.2496,39.116,0.0,68.9864,93.8785,96.0121,106.68
6,6,26563.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,280.0,280.0,210.0,0.0,0.0,134.039,131.112,157.023,161.828,60.7534,54.2441,148.267,108.488,177.197,55.6906,21.0436,101.979,72.3255,100.532,41.9488,39.779,0.0,70.1557,95.4696,97.6394,108.488
7,7,26375.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,280.0,280.0,210.0,0.0,0.0,146.517,153.67,141.566,159.927,74.9292,66.9011,182.863,133.802,0.0,0.0,230.14,0.0,89.2014,0.0,40.7944,0.0,94.5535,86.5254,117.746,120.422,133.802
8,8,29512.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,280.0,280.0,210.0,0.0,0.0,143.176,163.392,155.294,170.805,87.0799,77.7499,212.516,0.0,253.983,79.8232,0.0,0.0,103.666,144.096,60.1266,57.0166,0.0,31.1591,0.0,139.95,155.5
9,9,27427.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,280.0,280.0,210.0,0.0,0.0,163.297,144.48,172.638,185.318,96.1929,85.8865,0.0,171.773,280.563,88.1768,295.45,0.0,114.515,0.0,0.0,0.0,121.386,10.0175,0.0,0.0,171.773


In [41]:
#save result_df to csv
CSV.write("results/market_clearing_Jan.csv", result_df)

"results/market_clearing_Jan.csv"